# Eat Safe, Love

## Notebook Set Up

In [40]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint 
import pandas as pd

In [41]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [42]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [43]:
# review the collections in our database
db.list_collection_names()

['establishments']

In [44]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [77]:
# Find the establishments with a hygiene score of 20
query ={'scores.Hygiene': 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)

In [80]:
# Display the first document in the results using pprint
#passed in some fields just to make output smaller
pprint(establishments.find_one(query,{'BusinessName':1,"scores.Hygiene":1}))

{'BusinessName': 'The Chase Rest Home',
 '_id': ObjectId('63bdd273baf9ec662027cdb7'),
 'scores': {'Hygiene': 20}}


In [47]:
# Convert the result to a Pandas DataFrame
results = establishments.find(query)

# Display the number of rows in the DataFrame
hygiene_20 = pd.DataFrame(results)
# Display the first 10 rows of the DataFrame
hygiene_20.head(5)

,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,scores,SchemeType,geocode,RightToReply,Distance,NewRatingPending,meta,links,latitude,longitude
0,63bdd273baf9ec662027cdb7,110681,0,4029,The Chase Rest Home,Caring Premises,5,5-6 Southfields Road,Eastbourne,East Sussex,...,"{'Hygiene': 20, 'Structural': 20, 'ConfidenceI...",FHRS,"{'longitude': 0.27694, 'latitude': 50.769705}",,4613.888288,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings....",None,None
1,63bdd273baf9ec662027d13b,612039,0,1970/FOOD,Brenalwood,Caring Premises,5,Hall Lane,Walton-on-the-Naze,Essex,...,"{'Hygiene': 20, 'Structural': 15, 'ConfidenceI...",FHRS,"{'longitude': 1.278721, 'latitude': 51.857536}",,4617.965824,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings....",None,None
2,63bdd273baf9ec662027d445,730933,0,1698/FOOD,Melrose Hotel,Hotel/bed & breakfast/guest house,7842,53 Marine Parade East,Clacton On Sea,Essex,...,"{'Hygiene': 20, 'Structural': 20, 'ConfidenceI...",FHRS,"{'longitude': 1.15927, 'latitude': 51.789429}",,4619.656144,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings....",None,None
3,63bdd273baf9ec662027d631,172735,0,PI/000023858,Seaford Pizza,Takeaway/sandwich shop,7844,4 High Street,Seaford,East Sussex,...,"{'Hygiene': 20, 'Structural': 10, 'ConfidenceI...",FHRS,"{'longitude': 0.10202, 'latitude': 50.770885}",,4620.421725,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings....",None,None
4,63bdd273baf9ec662027d64b,172953,0,PI/000024532,Golden Palace,Restaurant/Cafe/Canteen,1,5 South Street,Seaford,East Sussex,...,"{'Hygiene': 20, 'Structural': 10, 'ConfidenceI...",FHRS,"{'longitude': 0.101446, 'latitude': 50.770724}",,4620.437179,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings....",None,None


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [72]:
#some of the RatingValues were strings, 
# converting those values toInt and excluding RatingValues with 'AwaitingInspection' --> kept getting a WriteError
query = {'LocalAuthorityName': {'$regex':'London'},'RatingValue':{'$ne':'AwaitingInspection'}}
establishments.update_many(query,[{'$set' : {'RatingValue': {'$toInt':'$RatingValue'}}}])

In [81]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
#query = {'LocalAuthorityName': {'$regex':'London'}, 'BusinessName':"Charlie's"}
query = {'LocalAuthorityName': {'$regex':'London'}, 'RatingValue': {'$gte':4}}

# Use count_documents to display the number of documents in the result
establishments.count_documents(query)

# Display the first document in the results using pprint
#passed in some fields to make output smaller
pprint(establishments.find_one(query,{'BusinessName':1, 'RatingValue':1, 'LocalAuthorityName':1}))


{'BusinessName': "Charlie's",
 'LocalAuthorityName': 'City of London Corporation',
 'RatingValue': 4,
 '_id': ObjectId('63bdd273baf9ec662027e7e2')}


In [76]:
# Convert the result to a Pandas DataFrame
results = list(establishments.find(query))

# Display the number of rows in the DataFrame
london_rating_gte_4_df = pd.DataFrame(results)

# Display the first 10 rows of the DataFrame
london_rating_gte_4_df.head(5)


,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,scores,SchemeType,geocode,RightToReply,Distance,NewRatingPending,meta,links,latitude,longitude
0,63bdd273baf9ec662027e7e2,621707,0,PI/000025307,Charlie's,Other catering premises,7841,Oak Apple Farm Building 103 Sheernes Docks,Sheppy Kent,,...,"{'Hygiene': 5, 'Structural': 10, 'ConfidenceIn...",FHRS,"{'longitude': 0.508551, 'latitude': 51.369321}",,4627.439468,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings....",None,None
1,63bdd273baf9ec662027eaf8,1130836,0,PI/000034075,Mv City Cruises Erasmus,Other catering premises,7841,Cherry Garden Pier,Cherry Garden Street Rotherhithe,Charlton,...,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': 0.249255999922752, 'latitude': 5...",,4627.873179,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...",None,None
2,63bdd273baf9ec662027f647,293783,0,PI/000002614,Benfleet Motor Yacht Club,Other catering premises,7841,Ferry Road,South Benfleet Essex,,...,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': 0.561954, 'latitude': 51.543831}",,4631.968418,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...",None,None
3,63bdd274baf9ec6620280448,1315095,0,PI/000036464,Coombs Catering t/a The Lock and Key,Restaurant/Cafe/Canteen,1,Leslie Ford House,Tilbury,Charlton,...,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': 0.346336990594864, 'latitude': 5...",,4636.846754,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...",None,None
4,63bdd274baf9ec6620280449,294474,0,PI/000014647,Tilbury Seafarers Centre,Restaurant/Cafe/Canteen,1,Tenants Row Tilbury Docks,Tilbury Essex,,...,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': 0.346325, 'latitude': 51.464078}",,4636.847650,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...",None,None


### 3. What are the top 5 establishments with a `RatingValue` rating value of '5', sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [91]:
#changing all RatingValue fields toInt, ignoring fields where 
# RatingValue = ['AwaitingInspection','AwaitingInspection','Exempt','AwaitingPublication','Awaiting Inspection','Pass'.]
query = {'RatingValue': {'$nin':['AwaitingInspection','Exempt','AwaitingPublication','Awaiting Inspection','Pass']}}
establishments.update_many(query,[{'$set': {'RatingValue': {'$toInt':'$RatingValue'}}}
])

In [51]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 
longitude = 

query = 
sort =  

# Print the results


SyntaxError: invalid syntax (848538274.py, line 6)

In [ ]:
# Convert result to Pandas DataFrame


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

# Print the first 10 results


In [ ]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame
